In [68]:
import numpy as np

from sklearn.metrics import mean_squared_error

In [69]:
result_dir = '../results/'

In [91]:
r_truth = (np.load(result_dir + 'truth/r_nottrained_truth.npy'))
t_truth = (np.load(result_dir + 'truth/scores_nottrained_truth.npy'))
cut1 = (t_truth[:,0]**2 + t_truth[:,1]**2 < 2500.)
cut2 = (np.log(r_truth)**2 < 10000.)
cut = cut1 & cut2

print(np.sum(cut1)/50000.)
print(np.sum(cut2)/50000.)
print(np.sum(cut)/50000.)

0.99884
1.0
0.99884


# Main table functions

In [72]:
table = ''
block_entries = []

In [77]:
indent = '   '
col_sep = ' & '
end_row = r'\\'
midrule = r'\midrule'
end_line = '\n'
header = (r'Algorithm & Setup'
          + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_1, \boldtheta_0) \right]$'
          + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$'
          + r' & $\MSE_\boldtheta \left[ \Delta E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$')
emphasis_begin = r'\mathbf{'
emphasis_end = r'}'

def init():
    global table, block_entries
    table = indent + header + end_row + end_line + indent + midrule + end_line
    block_entries = []
    
    
def finalise_block():
    global table, block_entries
    
    n_metrics = len(block_entries[0]) - 2
    
    # Find best performance
    block_metrics = [line[2:] for line in block_entries]
    block_metrics = np.array(block_metrics)
    block_best = [np.nanargmin(block_metrics[:,i]) for i in range(n_metrics)]
    
    # Format entries
    text = ''
    
    for i, line in enumerate(block_entries):
        text += indent + line[0] + col_sep + line[1] + col_sep
        for j in range(n_metrics):
            if np.isfinite(line[j + 2]):
                text += format_number(line[j + 2], 2, emphasize=(i == block_best[j]))
            if j == len(line) - 3:
                text += end_row + end_line
            else:
                text += col_sep
                
    # Add to document and reset for next block
    table += text
    block_entries = []
    
    
def new_block():
    global table, block_entries
    
    finalise_block()
    table += indent + midrule + end_line
    
    
def format_number(number,
                  precision=2,
                  trailing_zeros=True,
                  fix_minus_zero=True,
                  latex_math_mode=True,
                  emphasize=False):
    if precision == 0:
        temp =  str(int(round(number,precision)))
    elif trailing_zeros:
        temp =  ('{:.' + str(precision) + 'f}').format(round(number,precision))
    else:
        temp =  str(round(number,precision))
    if fix_minus_zero and len(temp) > 0:
        if temp[0] == '-' and float(temp) == 0.:
            temp = temp[1:]
    if latex_math_mode:
        if emphasize:
            temp = '$\mathbf{' + temp + '}$'
        else:
            temp = '$' + temp + '$'
    elif emphasize:
        temp = r'\emph{' + temp + r'}'
    return temp
    
    
def add(col1, col2, filename):
    global table, block_entries
          
    line = [col1, col2]
    
    # Calculate mean squared error for fixed theta pair
    try:
        log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
        log_r_estimated = np.log(np.load(result_dir + 'parameterized/r_nottrained_' + filename + '.npy'))
        line.append(mean_squared_error(log_r_truth[cut], log_r_estimated[cut]))
    except (IOError, ValueError):
        line.append(np.nan)
    
    # Calculate mean squared error over all theta
    try:
        expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
        expected_log_r_estimated = np.load(result_dir + 'parameterized/llr_' + filename + '.npy')
        line.append(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except (IOError, ValueError):
        line.append(np.nan)
    
    # Calculate mean squared error over all theta after subtracting minimum
    try:
        expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
        expected_log_r_truth -= np.min(expected_log_r_truth)
        expected_log_r_estimated = np.load(result_dir + 'parameterized/llr_' + filename + '.npy')
        expected_log_r_estimated -= np.min(expected_log_r_estimated)
        line.append(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except (IOError, ValueError):
        line.append(np.nan)
        
    block_entries.append(line)
    
    
def finalise():
    finalise_block()
    print(table)

# Table 1

In [78]:
labels = ['carl (raw)', 'carl (calibrated)',
          'score (raw)', 'score (calibrated)',
          'carl + score (raw)', 'carl + score (calibrated)',
          'regression (raw)', 'regression + score (raw)']

filenames = ['carl', 'carl_calibrated',
             'score', 'score_calibrated',
             'combined', 'combined_calibrated',
             'regression', 'combinedregression']

In [79]:
init()

for i, (l, f) in enumerate(zip(labels[:4], filenames[:4])):
    
    if i > 0:
        new_block()
        
    add(l, 'Baseline (2)', f + '_shallow')
    add('', 'Baseline (3)', f)
    add('', 'Baseline (5)', f + '_deep')
    
    add('', r'Random $\boldtheta$ (2)', f + '_random_shallow')
    add('', r'Random $\boldtheta$ (3)', f + '_random')
    add('', r'Random $\boldtheta$ (5)', f + '_random_deep')
    
    add('', 'Physics-aware (1)', f + '_aware_shallow')
    add('', 'Physics-aware (2)', f + '_aware')
    add('', 'Physics-aware (3)', f + '_aware_deep')

finalise()

   Algorithm & Setup & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_1, \boldtheta_0) \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$ & $\MSE_\boldtheta \left[ \Delta E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$\\
   \midrule
   carl (raw) & Baseline (2) & $\mathbf{0.08}$ & $\mathbf{1.27}$ & $1.16$\\
    & Baseline (3) & $0.14$ & $11.43$ & $1.60$\\
    & Baseline (5) & $0.20$ & $8.80$ & $1.73$\\
    & Random $\boldtheta$ (2) & $0.11$ & $3.12$ & $\mathbf{0.39}$\\
    & Random $\boldtheta$ (3) & $0.13$ & $4.96$ & $2.20$\\
    & Random $\boldtheta$ (5) & $0.25$ & $32.02$ & $1.23$\\
    & Physics-aware (1) &  &  & \\
    & Physics-aware (2) &  &  & \\
    & Physics-aware (3) &  &  & \\
   \midrule
   carl (calibrated) & Baseline (2) & $\mathbf{0.08}$ & $0.29$ & $0.31$\\
    & Baseline (3) & $0.11$ & $0.44$ & $0.54$\\
    & Baseline (5) & $0.19$ & $0.71$ & $1.14$\\
    & Random $\boldtheta$ (2) & $0.08$ & $\mathbf{0.24}$ & $\mathbf{0.26}$\\
  

/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:87: RuntimeWarning: invalid value encountered in log
/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


# Table 2

In [80]:
init()

for i, (l, f) in enumerate(zip(labels[4:], filenames[4:])):
    
    if i > 0:
        new_block()
        
    add(l, 'Baseline (2)', f + '_shallow')
    add('', 'Baseline (3)', f)
    add('', 'Baseline (5)', f + '_deep')
    
    add('', r'Random $\boldtheta$ (2)', f + '_random_shallow')
    add('', r'Random $\boldtheta$ (3)', f + '_random')
    add('', r'Random $\boldtheta$ (5)', f + '_random_deep')
    
    add('', 'Physics-aware (1)', f + '_aware_shallow')
    add('', 'Physics-aware (2)', f + '_aware')
    add('', 'Physics-aware (3)', f + '_aware_deep')

finalise()

   Algorithm & Setup & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_1, \boldtheta_0) \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$ & $\MSE_\boldtheta \left[ \Delta E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$\\
   \midrule
   carl + score (raw) & Baseline (2) & $\mathbf{0.03}$ & $\mathbf{1.62}$ & $\mathbf{0.23}$\\
    & Baseline (3) & $0.05$ & $6.66$ & $0.63$\\
    & Baseline (5) & $0.37$ & $92.43$ & $0.92$\\
    & Random $\boldtheta$ (2) & $0.15$ & $2.51$ & $2.99$\\
    & Random $\boldtheta$ (3) & $0.16$ & $2.29$ & $3.89$\\
    & Random $\boldtheta$ (5) & $0.19$ & $6.87$ & $2.76$\\
    & Physics-aware (1) &  &  & \\
    & Physics-aware (2) &  &  & \\
    & Physics-aware (3) &  &  & \\
   \midrule
   carl + score (calibrated) & Baseline (2) & $\mathbf{0.03}$ & $\mathbf{0.22}$ & $\mathbf{0.21}$\\
    & Baseline (3) & $0.04$ & $0.30$ & $0.31$\\
    & Baseline (5) & $0.06$ & $0.37$ & $0.28$\\
    & Random $\boldtheta$ (2) & $0.10$ & $0.86

/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:87: RuntimeWarning: invalid value encountered in log
/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


# Score table functions

In [81]:
table = ''
block_entries = []

In [64]:
indent = '   '
col_sep = ' & '
end_row = r'\\'
midrule = r'\midrule'
end_line = '\n'
header = (r'Algorithm & Setup'
          + r' & $\MSE_\boldx \left[ t_0(\boldx; \boldtheta_1, \boldtheta_0) \right]$'
          + r' & $\MSE_\boldx \left[ t_1(\boldx; \boldtheta_1, \boldtheta_0) \right]$')
emphasis_begin = r'\mathbf{'
emphasis_end = r'}'

def init():
    global table, block_entries
    table = indent + header + end_row + end_line + indent + midrule + end_line
    block_entries = []
    
    
def finalise_block():
    global table, block_entries
    
    n_metrics = len(block_entries[0]) - 2
    
    # Find best performance
    block_metrics = [line[2:] for line in block_entries]
    block_metrics = np.array(block_metrics)
    block_best = [np.nanargmin(block_metrics[:,i]) for i in range(n_metrics)]
    
    # Format entries
    text = ''
    
    for i, line in enumerate(block_entries):
        text += indent + line[0] + col_sep + line[1] + col_sep
        for j in range(n_metrics):
            if np.isfinite(line[j + 2]):
                text += format_number(line[j + 2], 1, emphasize=(i == block_best[j]))
            if j == len(line) - 3:
                text += end_row + end_line
            else:
                text += col_sep
                
    # Add to document and reset for next block
    table += text
    block_entries = []
    
    
def new_block():
    global table, block_entries
    
    finalise_block()
    table += indent + midrule + end_line
    
    
def format_number(number,
                  precision=2,
                  trailing_zeros=True,
                  fix_minus_zero=True,
                  latex_math_mode=True,
                  emphasize=False):
    if precision == 0:
        temp =  str(int(round(number,precision)))
    elif trailing_zeros:
        temp =  ('{:.' + str(precision) + 'f}').format(round(number,precision))
    else:
        temp =  str(round(number,precision))
    if fix_minus_zero and len(temp) > 0:
        if temp[0] == '-' and float(temp) == 0.:
            temp = temp[1:]
    if latex_math_mode:
        if emphasize:
            temp = '$\mathbf{' + temp + '}$'
        else:
            temp = '$' + temp + '$'
    elif emphasize:
        temp = r'\emph{' + temp + r'}'
    return temp
    
    
def add(col1, col2, filename):
    global table, block_entries
          
    line = [col1, col2]
    
    # Calculate mean squared error for fixed theta pair: component 0
    try:
        t_estimated = (np.load(result_dir + 'parameterized/scores_nottrained_' + filename + '.npy'))
        line.append(mean_squared_error(t_truth[cut], t_estimated[cut]))
    except (IOError, ValueError):
        line.append(np.nan)
        
    block_entries.append(line)
    
    
def finalise():
    finalise_block()
    print(table)

# Table 3

In [65]:
labels = ['carl (raw)',
          'score (raw)',
          'carl + score (raw)',
          'regression (raw)',
          'regression + score (raw)']

filenames = ['carl',
             'score',
             'combined',
             'regression',
             'combinedregression']

In [66]:
init()

for i, (l, f) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        new_block()
        
    add(l, 'Baseline (2)', f + '_shallow')
    add('', 'Baseline (3)', f)
    add('', 'Baseline (5)', f + '_deep')
    
    add('', r'Random $\boldtheta$ (2)', f + '_random_shallow')
    add('', r'Random $\boldtheta$ (3)', f + '_random')
    add('', r'Random $\boldtheta$ (5)', f + '_random_deep')
    
    add('', 'Physics-aware (1)', f + '_aware_shallow')
    add('', 'Physics-aware (2)', f + '_aware')
    add('', 'Physics-aware (3)', f + '_aware_deep')

finalise()

0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
0.0011600000000000499
   Algorit